### Importing Libraries

In [10]:
import numpy as np
import imutils
import cv2

### Initializing video-frame and reading template image

In [ ]:
vid = cv2.VideoCapture(0)
template = cv2.imread('template.png')

In [11]:

while(True):
    '''Starting loop for  capturing video frame'''   
    
    ret, frame = vid.read()
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # compute the Scharr gradient magnitude representation of the images
    # in both the x and y direction using OpenCV 2.4
    ddepth = cv2.cv.CV_32F if imutils.is_cv2() else cv2.CV_32F
    gradX = cv2.Sobel(gray, ddepth=ddepth, dx=1, dy=0, ksize=-1)
    gradY = cv2.Sobel(gray, ddepth=ddepth, dx=0, dy=1, ksize=-1)
    # subtract the y-gradient from the x-gradient
    gradient = cv2.add(gradX, gradY)
    gradient = cv2.convertScaleAbs(gradient)
    blurred = cv2.blur(gradient, (3, 3))
    (_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    closed = cv2.erode(closed, None, iterations = 1)
    closed = cv2.dilate(closed, None, iterations = 6)
    cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
            cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    if(len(cnts)!=0):
        c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
        # compute the rotated bounding box of the largest contour
        rect = cv2.minAreaRect(c)
        box = cv2.cv.BoxPoints(rect) if imutils.is_cv2() else cv2.boxPoints(rect)
        box = np.int0(box)
        # draw a bounding box arounded the detected barcode and display the
        # image
        cv2.drawContours(frame, [box], -1, (0, 255, 0), 3)
       
        result = cv2.matchTemplate(frame, template, cv2.TM_CCOEFF_NORMED)
    cv2.imshow("Image", closed)
    cv2.imshow("Image1", frame)
#     cv2.imshow("TemplateMatching",result)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# After the loop release the cap object
vid.release()
# Destroy all the windows
cv2.destroyAllWindows()


In [34]:
import numpy as np
from matplotlib import pyplot as plt
img = frame
img2 = img.copy()
template = cv2.imread('template.png')
result = cv2.matchTemplate(img, template, cv2.TM_CCOEFF_NORMED)